# High-level datasets

{py:class}`xarray_beam.Dataset` is new (as of September 2025) high-level interface for Xarray-Beam.

It requires less boilerplate code than the current (explicit) interface, and accordingly should be an easier to use tool, especially for non-expert users. You still need to think about how your data is divided into chunks, but the data model of `Dataset` keep track of the high-level structure of your data, avoiding the need to manually building templates in {py:class}`~xarray_beam.ChunksToZarr`.

```{warning}
The `Dataset` interface is experimental, and currently offers no backwards compatibility guarantees.
```

In [2]:
# small formatting improvements
import contextlib

@contextlib.contextmanager
def print_error():
  try:
    yield
  except Exception as e:
    print(f'{type(e).__name__}: {e}')

## Data model

Dataset is a wrapper over a series of Beam transformations, adding metadata describing the corresponding `xarray.Dataset` and how is distributed with Beam:

- `ptransform` is the wrapped `beam.PTransform` to compute the chunks of the dataset.
- `template` is a lazily-computed `xarray.Dataset` indicating the structure of the overall dataset.
- `chunks` is dictionary mapping from dimension names to integer chunk sizes, indicating the size of each chunk.
- `split_vars` is a boolean indicating if ptransform elements each contain only a single variable from the dataset, rather than all variables.

This information is surfaced via `xbeam.Dataset.__repr__()`:

In [3]:
import apache_beam as beam
import xarray_beam as xbeam
import xarray
import numpy as np
import pandas as pd

xarray_ds = xarray.Dataset(
    {'temperature': (('time', 'longitude', 'latitude'), np.random.randn(365, 180, 90))},
    coords={'time': pd.date_range('2025-01-01', freq='1D', periods=365)},
)
chunks = {'time': 100, 'longitude': 90, 'latitude': 90}
xbeam_ds = xbeam.Dataset.from_xarray(xarray_ds, chunks)
xbeam_ds

Xarray-Beam pipelines typically read and write data to [Zarr](https://zarr.dev/), so we'll start by writing our example data to a Zarr file (with plain Xarray/Dask) for us to read later:

In [4]:
xarray_ds.chunk(chunks).to_zarr('example_data.zarr', mode='w')

## Writing pipelines

Most Xarray-Beam pipelines can be written via a handful of Dataset methods:

- {py:meth}`~xarray_beam.Dataset.from_zarr`: Load a dataset from a Zarr store.
- {py:meth}`~xarray_beam.Dataset.rechunk`: Adjust chunks on a dataset.
- {py:meth}`~xarray_beam.Dataset.map_blocks`: Map a function over every chunk of this dataset independently.
- {py:meth}`~xarray_beam.Dataset.to_zarr`: Write a dataset to a Zarr store.

All non-trivial computation happens via the embarrasingly parallel `map_blocks` method.

In order for `map_blocks` to work, data needs to be appropriately chunked. Here are a few typical chunking patterns that work well for most needs:

- "Pencil" chunks, which group together all times, and parallelize over space. These long and skinny chunks look like a box of pencils:

In [5]:
xarray_ds.temperature.chunk({'time': -1, 'latitude': 20, 'longitude': 20}).data

- "Pancake" chunks, which group together all spatial locations, and parallelize over time. These flat and wide chunks look like a stack of pancakes:

In [6]:
xarray_ds.temperature.chunk({'time': 1, 'latitude': -1, 'longitude': -1}).data

Weather/climate datasets are typically generated and stored in pancake chunks, but pencil chunks are more useful for most analytics queries, which requires large histories of weather at a single location. Intermediate "compromise" chunks can sometimes be a good idea, although if performance and flexibility are critical it may be worth storing multiple copies of your data in different formats.

Using the right chunks is *absolutely essentially* for efficient operations with Xarray-Beam and Zarr. For example, reading data from a single location across all times (a "pencil" query) is extremely inefficient for a dataset stored in "pancake" chunks -- it would require loading the entire dataset from disk!

Rechunking is a fundamentally an expensive operation (it requires multiple complete reads/writes of a dataset from disk), but in Xarray-Beam it's straightforward, via {py:meth}`~xarray_beam.Dataset.rechunk`.

### Example 1: Climatology

Here we need to group together all time points in the same chunk ("pencil chunks"), parallelizing over space:

In [7]:
with beam.Pipeline() as p:
  p | (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .rechunk({'time': -1, 'latitude': 30, 'longitude': 30})
      .map_blocks(lambda ds: ds.groupby('time.month').mean())
      .to_zarr('example_climatology.zarr')
  )
xarray.open_zarr('example_climatology.zarr')

### Example 2: Regridding over space

Here we need to group all space points in the same chunk ("pancake chunks"), parallelizing over time:

In [8]:
with beam.Pipeline() as p:
  p | (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .rechunk({'time': 10, 'latitude': -1, 'longitude': -1})
      .map_blocks(lambda ds: ds.coarsen(latitude=2, longitude=2).mean())
      .to_zarr('example_regrid.zarr')
  )
xarray.open_zarr('example_regrid.zarr')

## Limitations of map_blocks

In the examples above, {py:meth}`~xarray_beam.Dataset.map_blocks` somehow automatically knew the appropriate structure of the output `template`, without evaluating any chunked data. How could this work?

For building templates, Xarray-Beam relies on lazy evaluation with [Dask arrays](https://docs.dask.org/en/stable/array.html). This requires that applied functions are Dask compatible. Almost all built-in Xarray operations are Dask compatible, but if your applied function is _not_ Dask compatible (e.g., because it loads array values into memory), Xarray-Beam will show an informative error:

In [9]:
with print_error():
  (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .map_blocks(lambda ds: ds.compute())  # load into memory
  )

You can avoid these errors by explicitly [creating a template](creating_templates):

In [10]:
ds_beam = xbeam.Dataset.from_zarr('example_data.zarr')
ds_beam.map_blocks(lambda ds: ds.compute(), template=ds_beam.template)

In other situations, you might want to perform an operation that returns something other than an `xarray.Dataset`, e.g., to write all chunks as individual files to disk. In these situations, you can switch to the lower-level Xarray-Beam [data model](data-model), and use raw Beam operations:

In [14]:
def to_netcdf(key: xbeam.Key, chunk: xarray.Dataset):
  path = f"{chunk.indexes['time'][0]:%Y-%m-%d}.nc"
  chunk.to_netcdf(path)

with beam.Pipeline() as p:
  p | (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .rechunk({'latitude': -1, 'longitude': -1})
      .ptransform
  ) | beam.MapTuple(to_netcdf)

%ls *.nc